In [19]:
from langchain import hub
from langchain_core.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.runnables import RunnableConfig
import query_tool
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain.memory import ConversationBufferMemory


In [21]:
tools = [query_tool.reference_code_search]


prompt: ChatPromptTemplate = hub.pull("hwchase17/openai-tools-agent") 

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are an AWS solutions architect. Your job is to help customers build and deploy secure applications on AWS. Follow architecture best practices and find out approriate requirements before suggesting a solution. Only use tools once necessary for generating the output."),
  ("human", "My current terraform file:\n{tf_file}"),
  MessagesPlaceholder("chat_history", optional=True),
  ("human", "{input}"),
  "{agent_scratchpad}"
])


llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=.5)
# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt) # type: ignore # wrong type for query_tool import

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # type: ignore # wrong type for query_tool import

In [22]:
from fastapi import concurrency


agent_executor.invoke({
  # "input": "Can you help me set up a server and a database?",
  "input": "Which instance type is my tf file currently using?",
  "tf_file": "```terraform resource \"aws_instance\" \"example\" { ami = \"${data.aws_ami.ubuntu.id}\" instance_type = \"t2.micro\" }```",
  "chat_history": [
    HumanMessage(content="hi! my name is bob"),
    # AIMessage(content="Hello Bob! How can I assist you today?"),
  ],
}, RunnableConfig(
    recursion_limit=2
  )
)




> Entering new AgentExecutor chain...

Invoking: `reference_code_search` with `{'query': 't2.micro'}`


# Configure the AWS provider
provider "aws" {
  region = "eu-west-1"
}

# Create a DB instance
resource "aws_db_instance" "example" {
  engine              = "mysql"
  allocated_storage   = 10
  instance_class      = "db.t2.micro"
  name                = "example_database_stage"
  username            = "admin"
  password            = "${var.db_password}"
  skip_final_snapshot = true
}

Invoking: `reference_code_search` with `{'query': 'aws_instance'}`


# Configure the AWS provider
provider "aws" {
  region = "eu-west-1"
}

# Create a DB instance
resource "aws_db_instance" "example" {
  engine              = "mysql"
  allocated_storage   = 10
  instance_class      = "db.t2.micro"
  name                = "example_database_stage"
  username            = "admin"
  password            = "${var.db_password}"
  skip_final_snapshot = true
}

Invoking: `reference_code_search` with `{'query

{'input': 'Which instance type is my tf file currently using?',
 'tf_file': '```terraform resource "aws_instance" "example" { ami = "${data.aws_ami.ubuntu.id}" instance_type = "t2.micro" }```',
 'chat_history': [HumanMessage(content='hi! my name is bob')],
 'output': 'Agent stopped due to max iterations.'}